In [61]:
from sklearn.datasets import load_digits
digits = load_digits()

print(digits.data.shape)
print(digits.target[44])

(1797, 64)
7


In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

In [63]:
data_tensor = torch.Tensor(digits.data)
target_tensor = F.one_hot(torch.Tensor(digits.target).long(), 10).float()

dataset = TensorDataset(data_tensor, target_tensor)

# Create dataloader
# Wait we are not doing backpropogation so cant we just make batch_size the entire thing?
batch_size = 1797
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

NEAT PYTORCH

to do
genome species population stuff
crossover
improve parallelization i think

Parallelize the eval of each nn. The nn is not parallelizable because mm cannot be used. 


rn every new connection gets an innovation number. this is wrong. you should check if they exist. if they exists you have to copy that innov number

In [69]:
# NEAT Classes

from collections import deque, defaultdict
import random

torch.manual_seed(42)

class Node:
    def __init__(self, id_, input_=False, output=False):
        self.id = id_
        self.is_input = input_
        self.is_output = output

        self.val = torch.Tensor(0) # Tensors
        self.num_incoming_connections = 0
        
        if not input_ and not output:
            # Output has special handling when init because it starts off fully connected
            # These are nodes that are created from splitting connection
            self.num_incoming_connections = 1
            
        self.received = 0 # Keep track of nodes received before applying activation function
        
class ConnectionGene:
    def __init__(self, in_node, out_node, innov_num, weight):
        self.in_node = in_node
        self.out_node = out_node
        self.innov_num = innov_num
        
        self.weight = weight # Weights are tensors
        self.enable = True # If node is disabled, it CAN be reenabled

        self.resulting_node = None # If this connection is split, what is the resulting node

class NN(nn.Module):
    global_nodes = []

    # Existing connections
    # key: tuple of in and out node
    # value: connection object
    # Index is innov_number
    # Now that we use this schema, might have to do some crazy refactoring
    global_connections = {}
    
    def __init__(self, input_dim, output_dim, mutate=False, crossover=False):
        super(NN, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.nodes = [] # Nodes in this specific NN
        self.connections = [] # Connections in this specific NN

        # Mutated models should not be inited!
        if mutate:
            return
        elif crossover:
            # Initialized in a different way
            self.input_dim = input_dim
            self.output_dim = output_dim
            
            self.nodes = []
            
            self.connections = []
        else:
            # When init a new model, the node nums and innov should be the same as any other new initialized model
            # This else only gets models in the initial population
            # Initalize a fully connected NN with no hidden layers
            for i in range(input_dim):
                if len(NN.global_nodes) <= i:
                    new_node = Node(True)
                    NN.global_nodes.append(new_node)
                self.nodes.append(i)
    
            for i in range(output_dim):
                node_index = i + input_dim
                if len(NN.global_nodes) <= node_index:
                    new_node = Node(False, True)
                    NN.global_nodes.append(new_node)
                self.nodes.append(node_index)

                for j in range(input_dim):
                    if (j, node_index) not in NN.global_connections.keys():
                        conn = ConnectionGene(NN.global_nodes[j], NN.global_nodes[node_index], torch.randn(1))
                        NN.global_connections.update({(j, node_index): conn})
                    self.connections.append(list(NN.global_connections.keys()).index((j, node_index)))
                    NN.global_nodes[node_index].num_incoming_connections += 1 # Each output starts off fully connected to input

    def reset(self):
        for i in self.nodes:
            NN.global_nodes[i].val = 0
            NN.global_nodes[i].received = 0

    def clone(self, mutate=False, crossover=False):
        # Create a new NN instance with same input/output dims
        new_nn = NN(self.input_dim, self.output_dim, mutate, crossover)
        
        # Deep copy nodes
        new_nn.nodes = []
        for node in self.nodes:
            new_node = Node(node.id, node.is_input, node.is_output)
            new_node.val = node.val
            new_node.num_incoming_connections = node.num_incoming_connections
            new_node.received = node.received
            new_nn.nodes.append(new_node)

        # Deep copy connections
        new_nn.connections = []
        for conn in self.connections:
            # Find the corresponding new nodes by id
            in_node = next(n for n in new_nn.nodes if n.id == conn.in_node.id)
            out_node = next(n for n in new_nn.nodes if n.id == conn.out_node.id)

            new_conn = ConnectionGene(in_node, out_node, conn.weight, conn.innov_num)
            new_conn.enable = conn.enable
            new_nn.connections.append(new_conn)

        return new_nn

    def forward(self, x: torch.Tensor):
        if x.shape[1] != self.input_dim:
            raise ValueError("Input dim is not correct")
        
        batch_size = x.shape[0]
        self.reset()
    
        # Create batch versions of node values and received counts
        for node in self.nodes:
            node.vals = torch.zeros(batch_size)
            node.received = torch.zeros(batch_size, dtype=torch.int)
    
        # Set input values
        for idx in range(self.input_dim):
            self.nodes[idx].vals = x[:, idx]
    
        # Start with nodes whose incoming connections are already satisfied
        queue = deque()
        for node in self.nodes:
            if node.num_incoming_connections == 0:
                queue.append(node)
    
        while queue:
            curr_node = queue.popleft()
    
            for conn in self.connections:
                if not conn.enable:
                    continue
                if conn.in_node != curr_node:
                    continue
    
                out_node = conn.out_node
                out_node.vals += curr_node.vals * conn.weight
                out_node.received += 1
    
                # Only enqueue if all inputs are received
                # Note: vectorized check — adds node to queue if all samples are ready
                if (out_node.received == out_node.num_incoming_connections).all():
                    if not out_node.is_output:
                        out_node.vals = torch.relu(out_node.vals)
                    queue.append(out_node)
    
        # Collect logits from output nodes
        output_vals = [node.vals for node in self.nodes if node.is_output]
        logits = torch.stack(output_vals, dim=1)  # shape: (batch_size, num_outputs)
        return logits
    
    def weight_perturbation(self, quiet):
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()
        
        mean = 0.0
        std_dev = 0.1
        
        noise = torch.randn_like(self.connections[rand_conn_id].weight) * std_dev + mean
        self.connections[rand_conn_id].weight += noise
        
        if not quiet:
            print(f'connection {rand_conn_id} weight perturbated to {self.connections[rand_conn_id].weight.item():.2f}')
        
    def weight_modification(self, quiet):
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        self.connections[rand_conn_id].weight = torch.randn(1)
        
        if not quiet:
            print(f'connection {rand_conn_id} weight modified to {self.connections[rand_conn_id].weight.item():.2f}')

    def add_connection(self, quiet):
        max_attempts = 100  # prevent infinite loop
        for _ in range(max_attempts):
            rand_node_in = self.nodes[torch.randint(0, len(self.nodes), (1,)).item()]
        
            if rand_node_in.is_output:
                continue
        
            rand_node_out = self.nodes[torch.randint(0, len(self.nodes), (1,)).item()]
        
            if rand_node_out == rand_node_in or rand_node_out.is_input:
                continue
        
            # Skip if connection already exists
            if any(conn.in_node == rand_node_in and conn.out_node == rand_node_out for conn in self.connections):
                continue
        
            # Optional: skip if this would form a cycle
            # if self.creates_cycle(rand_node_in, rand_node_out):
            #     continue
        
            conn = ConnectionGene(rand_node_in, rand_node_out, torch.randn(1), NN.glob_innov_num)
            self.connections.append(conn)
            NN.glob_innov_num += 1
            rand_node_out.num_incoming_connections += 1
            
            if not quiet:
                print(f"Connection created from node {rand_node_in.id} to node {rand_node_out.id} (innovation #{NN.glob_innov_num})")

            return
            
        if not quiet:
            print("Failed to add connection after max attempts.")

    def add_node(self, quiet):            
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        while not self.connections[rand_conn_id].enable:
            rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        # Splits an existing connection by adding a node
        self.connections[rand_conn_id].enable = False
        
        new_node = Node(NN.glob_node_num)
        self.nodes.append(new_node)
        NN.glob_node_num += 1
        
        conn = ConnectionGene(self.connections[rand_conn_id].in_node, new_node, torch.randn(1), NN.glob_innov_num)
        self.connections.append(conn)
        NN.glob_innov_num += 1
        
        conn = ConnectionGene(new_node, self.connections[rand_conn_id].out_node, torch.randn(1), NN.glob_innov_num)
        self.connections.append(conn)
        NN.glob_innov_num += 1
        
        if not quiet:
            print(f'connection {rand_conn_id} split')

    def toggle_connection(self, quiet):
        rand_conn_id = torch.randint(0, len(self.connections), (1,)).item()

        self.connections[rand_conn_id].enable = not self.connections[rand_conn_id].enable

        if not quiet:
            print(f'connection {rand_conn_id} toggled to {self.connections[rand_conn_id].enable}')

    def mutate(self, quiet=False):
        # For this experiment i use 60 weight perturbation, 60 weight mutation, 5 add connection, 3 add node, 2 toggle
        # Each one is chosen independently of each other
        # Does not include crossover. That cannot be done by itself
        new_model = self.clone(True)

        # Do mutations on new_model
        if random.random() < 0.60:
            new_model.weight_perturbation(quiet)
        if random.random() < 0.60:
            new_model.weight_modification(quiet)
        if random.random() < 0.05:
            new_model.add_connection(quiet)
        if random.random() < 0.03:
            new_model.add_node(quiet)
        if random.random() < 0.02:
            new_model.toggle_connection(quiet)
    
        return new_model

In [70]:
def crossover(info1, info2):
    # Equal Fitness
    # Might not implement this rn because it doesnt happen that much
    if info1['loss'] == info2['loss']:
        pass

    # Find fitter model
    fit_model, less_fit_model = (info2['model'], info1['model']) if info1['loss'] > info2['loss'] else (info1['model'], info2['model'])

    # New model starts off as clone of more fit
    new_model = fit_model.clone(True)

    fit_conn_pointer = less_fit_conn_pointer = 0

    while fit_conn_pointer != len(fit_model.connections) - 1 and less_fit_conn_pointer != len(less_fit_model.connections) - 1:
        # IDK

    return new_model

IndentationError: expected an indented block after 'while' statement on line 15 (1054849484.py, line 18)

In [71]:
def measure_compatibility(genome1, genome2, c1, c2, c3, delta_thresh):
    genome1_conns = {i.innov_num: i.weight for i in genome1.connections}
    genome2_conns = {i.innov_num: i.weight for i in genome2.connections}

    innovs1 = set(genome1_conns.keys())
    innovs2 = set(genome2_conns.keys())

    max_innov1 = max(innovs1) if innovs1 else 0
    max_innov2 = max(innovs2) if innovs2 else 0
    max_innov = max(max_innov1, max_innov2)

    # Matching genes: innovation numbers in both genomes
    matching = innovs1.intersection(innovs2)
    # Calculate average weight difference for matching genes
    if matching:
        weight_diff = sum(abs(genome1_conns[i] - genome2_conns[i]) for i in matching) / len(matching)
    else:
        weight_diff = 0

    # Excess genes: genes whose innovation number is greater than max innovation number of other genome
    excess = 0
    for innov in innovs1:
        if innov > max_innov2:
            excess += 1
    for innov in innovs2:
        if innov > max_innov1:
            excess += 1

    # Disjoint genes: genes that do not match and are not excess
    disjoint = (len(innovs1 - innovs2) + len(innovs2 - innovs1)) - excess

    # Normalization factor N
    N = max(len(genome1_conns), len(genome2_conns))
    if N < 20:
        N = 1  # as per original NEAT paper for small genomes

    delta = (c1 * excess / N) + (c2 * disjoint / N) + (c3 * weight_diff)
    
    # return delta
    return True if delta < delta_thresh else False

In [72]:
model_infos = [{"model": NN(64, 10), "loss": float('inf')} for _ in range(10)]
# top_models = [info["model"] for info in model_infos[:5]]

# for model in top_models:
#         model_infos.append({"model": model.mutate(), "loss": float('inf')})


In [60]:
# "Training loop"
import math

# Hyperparameters
population_size = 100
epochs = 500
input_dim = 64
output_dim = 10
top_k = 0.2 # The percentage of genomes to keep for reproduction

# hyperparameters for measuring compatibility from https://nn.cs.utexas.edu/downloads/papers/stanley.cec02.pdf
c1 = 1.0
c2 = 1.0
c3 = 3.0
delta_thresh = 4.0

# Init
# Using list of lists
# Dead species will not be kept track of. There will be no empty list
population = []

for _ in range(population_size):
    new_model = {"model": NN(input_dim, output_dim), "loss": float('inf'), "fitness": -float('inf')}

    # First model
    if len(population) == 0:
        population.append([new_model])
    else:
        for idx, species in enumerate(population):
            if measure_compatibility(new_model['model'], species[0]['model'], c1, c2, c3, delta_thresh):
                population[idx].append(new_model)
            else:
                # New species created
                population.append([new_model])
                
loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print(f"epoch: {epoch}")
    
    for species in population:
        for model_info in species:
            model_info["model"].reset()
        
        with torch.no_grad():
            for model_info in species:
                model = model_info["model"]
                total_loss = 0.0
                total_samples = 0
    
                for data_batch, label_batch in loader:
                    output = model(data_batch)
                    loss = loss_fn(output, label_batch)
                    total_loss += loss.item() * data_batch.size(0)
                    total_samples += data_batch.size(0)
                
                model_info["loss"] = total_loss / total_samples

    flattened_population = []

    for species in population:
        for genome in species:
            flattened_population.append(genome)
            
    ranked_models = sorted([model_info for model_info in flattened_population], key=lambda x: x["loss"])
    print(f"top model loss: {ranked_models[0]['loss']:.2f}")

    # Fitness sharing
    for species in population:
        species_size = len(species)
        for genome in species:
            raw_fitness = 1 / (1 + genome['loss'])
            genome['fitness'] = raw_fitness / species_size

    # This is just a list not a list of lists
    new_population = []

    for species in population:
        offspring = []
        
        ranked_models = sorted([model_info for model_info in species], key=lambda x: x["fitness"], reverse=True)
        parents = [model_info['model'] for model_info in ranked_models[:math.ceil(top_k * len(ranked_models))]]

        while len(offspring) != len(ranked_models):
            offspring.append({"model": random.choice(parents).mutate(True), "loss": float('inf'), "fitness": -float('inf')})
            
        new_population.extend(offspring)

    # Redivide into species
    new_population_divided = []

    for model in new_population:    
        # First model
        if len(new_population_divided) == 0:
            new_population_divided.append([model])
        else:
            for idx, species in enumerate(new_population_divided):
                if measure_compatibility(model['model'], species[0]['model'], c1, c2, c3, delta_thresh):
                    new_population_divided[idx].append(model)
                else:
                    # New species created
                    new_population_divided.append([model])
                    
    population = new_population_divided
        

epoch: 0
top model loss: 49.73
epoch: 1
top model loss: 49.39
epoch: 2
top model loss: 46.70
epoch: 3
top model loss: 44.24
epoch: 4
top model loss: 44.56
epoch: 5
top model loss: 42.57
epoch: 6
top model loss: 41.27
epoch: 7
top model loss: 40.95
epoch: 8
top model loss: 40.61
epoch: 9
top model loss: 40.59
epoch: 10
top model loss: 38.80
epoch: 11
top model loss: 36.54
epoch: 12
top model loss: 36.41
epoch: 13
top model loss: 36.33
epoch: 14
top model loss: 35.70
epoch: 15
top model loss: 35.41
epoch: 16
top model loss: 35.19
epoch: 17
top model loss: 35.17
epoch: 18
top model loss: 35.00
epoch: 19
top model loss: 34.77
epoch: 20
top model loss: 33.76
epoch: 21
top model loss: 33.78
epoch: 22
top model loss: 33.15
epoch: 23
top model loss: 32.50
epoch: 24
top model loss: 31.95
epoch: 25
top model loss: 31.61
epoch: 26
top model loss: 31.53
epoch: 27
top model loss: 31.19
epoch: 28
top model loss: 30.15
epoch: 29
top model loss: 30.02
epoch: 30
top model loss: 30.30
epoch: 31
top mode

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103da63c0>>
Traceback (most recent call last):
  File "/Users/paullin/Documents/neuroevol/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


top model loss: 3.19
epoch: 347


KeyboardInterrupt: 

In [101]:
model = NN(64, 10)

In [96]:
model(data_tensor[0])

tensor([ -8.2188, -15.4980, -83.0724,  33.3792, -99.2694,   3.3089,  24.0452,
        -37.0231, -81.9885,  47.4248])

In [102]:
model.mutate()

connection 597 weight perturbated to 1.89


NN()

In [ ]:
data_tensor[0]

In [ ]:
model(data_tensor[0])